# 🧪 Week 1 Experiments: ML & Quantum Fundamentals

**Objective**: Hands-on exploration of machine learning and quantum computing concepts

**Time Allocation**: 2 hours total
- Experiment 1: Data Exploration (45 mins)
- Experiment 2: Simple ML Baseline (45 mins) 
- Experiment 3: Quantum Circuit Simulation (30 mins)

**Learning Goals**:
- Understand parking data characteristics
- Build your first ML pricing model
- Create and run quantum circuits
- Connect theory to practice

## Setup & Imports

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Quantum Computing
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute, Aer
from qiskit.visualization import plot_histogram, plot_circuit_layout

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✅ All imports successful!")
print(f"📊 NumPy: {np.__version__}")
print(f"🐼 Pandas: {pd.__version__}")
print(f"🚀 Ready to explore!")

---
# 🔍 Experiment 1: Data Exploration (45 minutes)

**Goal**: Understand parking data patterns and create synthetic dataset for learning

## 1.1 Create Synthetic Parking Dataset

In [ ]:
def generate_parking_data(n_samples=1000, n_locations=5):
    """
    Generate synthetic parking data with realistic patterns
    
    Features:
    - Temporal: hour, day_of_week, is_weekend
    - Demand: occupancy_rate, queue_length  
    - External: weather_score, event_factor
    - Location: location_id, base_price
    """
    
    # Generate time-based features
    hours = np.random.randint(6, 23, n_samples)  # 6 AM to 10 PM
    days = np.random.randint(0, 7, n_samples)    # Monday=0 to Sunday=6
    is_weekend = (days >= 5).astype(int)
    
    # Generate location data
    location_ids = np.random.randint(0, n_locations, n_samples)
    location_names = [f"Zone_{i}" for i in range(n_locations)]
    
    # Create demand patterns based on time
    # Rush hours (8-9 AM, 5-7 PM) have higher demand
    rush_hour_morning = ((hours >= 8) & (hours <= 9)).astype(int)
    rush_hour_evening = ((hours >= 17) & (hours <= 19)).astype(int)
    
    # Base occupancy influenced by time patterns
    base_occupancy = 0.3 + 0.4 * (rush_hour_morning + rush_hour_evening)
    
    # Add weekend effects (different patterns)
    weekend_effect = is_weekend * 0.2 * np.random.normal(0, 1, n_samples)
    
    # Final occupancy with noise
    occupancy_rate = np.clip(
        base_occupancy + weekend_effect + np.random.normal(0, 0.15, n_samples),
        0.05, 0.95
    )
    
    # Queue length correlated with occupancy
    queue_length = np.maximum(0, 
        np.round(5 * occupancy_rate + np.random.exponential(1, n_samples))
    ).astype(int)
    
    # External factors
    weather_score = np.random.uniform(0.3, 1.0, n_samples)  # 1.0 = perfect weather
    event_factor = np.random.choice([0, 1], n_samples, p=[0.85, 0.15])  # 15% chance of events
    
    # Generate prices based on demand (this is what we want to predict/optimize)
    base_prices = np.array([8, 10, 12, 15, 18])  # Different zones have different base prices
    zone_base_price = base_prices[location_ids]
    
    # Price influenced by occupancy, time, and external factors
    price_multiplier = (
        1.0 +                                    # Base
        0.8 * occupancy_rate +                   # Occupancy effect
        0.3 * (rush_hour_morning + rush_hour_evening) +  # Rush hour premium
        0.2 * event_factor +                     # Event premium
        0.1 * (1 - weather_score)                # Bad weather premium
    )
    
    optimal_price = zone_base_price * price_multiplier
    
    # Create DataFrame
    data = pd.DataFrame({
        'location_id': location_ids,
        'location_name': [location_names[i] for i in location_ids],
        'hour': hours,
        'day_of_week': days,
        'is_weekend': is_weekend,
        'occupancy_rate': np.round(occupancy_rate, 3),
        'queue_length': queue_length,
        'weather_score': np.round(weather_score, 2),
        'event_factor': event_factor,
        'base_price': zone_base_price,
        'optimal_price': np.round(optimal_price, 2)
    })
    
    return data

# Generate our dataset
print("🔄 Generating synthetic parking data...")
parking_data = generate_parking_data(n_samples=2000, n_locations=5)
print(f"✅ Generated {len(parking_data)} parking records")
print(f"📊 Shape: {parking_data.shape}")

# Display first few rows
parking_data.head()

## 1.2 Exploratory Data Analysis

In [ ]:
# Basic statistics
print("📈 Dataset Overview:")
print(f"Total samples: {len(parking_data)}")
print(f"Features: {len(parking_data.columns)}")
print(f"Locations: {parking_data['location_name'].nunique()}")
print(f"Price range: ${parking_data['optimal_price'].min():.2f} - ${parking_data['optimal_price'].max():.2f}")
print(f"Avg occupancy: {parking_data['occupancy_rate'].mean():.1%}")

# Check for missing values
print("\n🔍 Missing Values:")
print(parking_data.isnull().sum())

# Statistical summary
print("\n📊 Statistical Summary:")
parking_data.describe()

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('🔍 Parking Data Exploration', fontsize=16, fontweight='bold')

# 1. Price distribution
axes[0,0].hist(parking_data['optimal_price'], bins=30, alpha=0.7, color='skyblue')
axes[0,0].set_title('Price Distribution')
axes[0,0].set_xlabel('Price ($)')
axes[0,0].set_ylabel('Frequency')

# 2. Occupancy vs Price
axes[0,1].scatter(parking_data['occupancy_rate'], parking_data['optimal_price'], 
                  alpha=0.6, color='coral')
axes[0,1].set_title('Occupancy vs Price')
axes[0,1].set_xlabel('Occupancy Rate')
axes[0,1].set_ylabel('Price ($)')

# 3. Hourly demand patterns
hourly_avg = parking_data.groupby('hour')['occupancy_rate'].mean()
axes[0,2].plot(hourly_avg.index, hourly_avg.values, marker='o', color='green')
axes[0,2].set_title('Average Occupancy by Hour')
axes[0,2].set_xlabel('Hour of Day')
axes[0,2].set_ylabel('Avg Occupancy Rate')
axes[0,2].grid(True, alpha=0.3)

# 4. Price by location
location_prices = parking_data.groupby('location_name')['optimal_price'].mean().sort_values()
axes[1,0].bar(range(len(location_prices)), location_prices.values, color='purple', alpha=0.7)
axes[1,0].set_title('Average Price by Location')
axes[1,0].set_xlabel('Location')
axes[1,0].set_ylabel('Avg Price ($)')
axes[1,0].set_xticks(range(len(location_prices)))
axes[1,0].set_xticklabels(location_prices.index, rotation=45)

# 5. Weekend vs Weekday
weekend_comparison = parking_data.groupby('is_weekend')[['occupancy_rate', 'optimal_price']].mean()
x = np.arange(2)
width = 0.35
axes[1,1].bar(x - width/2, weekend_comparison['occupancy_rate'], width, 
              label='Occupancy Rate', alpha=0.8)
axes[1,1].bar(x + width/2, weekend_comparison['optimal_price']/20, width, 
              label='Price (scaled)', alpha=0.8)
axes[1,1].set_title('Weekend vs Weekday Patterns')
axes[1,1].set_ylabel('Value')
axes[1,1].set_xticks(x)
axes[1,1].set_xticklabels(['Weekday', 'Weekend'])
axes[1,1].legend()

# 6. Correlation heatmap
numeric_cols = ['hour', 'day_of_week', 'occupancy_rate', 'queue_length', 
                'weather_score', 'event_factor', 'optimal_price']
correlation_matrix = parking_data[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            ax=axes[1,2], cbar_kws={'shrink': 0.8})
axes[1,2].set_title('Feature Correlations')

plt.tight_layout()
plt.show()

print("\n🎯 Key Insights:")
print(f"• Strongest price correlation: {correlation_matrix['optimal_price'].abs().sort_values(ascending=False).index[1]} ({correlation_matrix['optimal_price'].abs().sort_values(ascending=False).iloc[1]:.3f})")
print(f"• Peak occupancy hour: {hourly_avg.idxmax()}:00 ({hourly_avg.max():.1%})")
print(f"• Most expensive location: {location_prices.index[-1]} (${location_prices.iloc[-1]:.2f})")

---
# 🤖 Experiment 2: Simple ML Baseline (45 minutes)

**Goal**: Build and evaluate a basic machine learning model for price prediction

## 2.1 Prepare Data for Machine Learning

In [ ]:
# Select features for modeling
feature_columns = [
    'hour', 'day_of_week', 'is_weekend',
    'occupancy_rate', 'queue_length', 
    'weather_score', 'event_factor',
    'location_id'  # We'll use location as a categorical feature
]

target_column = 'optimal_price'

# Prepare features and target
X = parking_data[feature_columns].copy()
y = parking_data[target_column].copy()

print(f"📊 Features shape: {X.shape}")
print(f"🎯 Target shape: {y.shape}")
print(f"\n🔧 Feature columns:")
for i, col in enumerate(feature_columns, 1):
    print(f"  {i}. {col}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=X['location_id']
)

print(f"\n✂️ Data Split:")
print(f"• Training set: {X_train.shape[0]} samples")
print(f"• Testing set: {X_test.shape[0]} samples")
print(f"• Split ratio: {X_test.shape[0]/X.shape[0]:.1%} test")

## 2.2 Train Linear Regression Model

In [ ]:
# Create and train the model
print("🔄 Training Linear Regression model...")
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("✅ Model training complete!")

# Calculate performance metrics
def calculate_metrics(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n📊 {dataset_name} Performance:")
    print(f"• MAE (Mean Absolute Error): ${mae:.2f}")
    print(f"• RMSE (Root Mean Square Error): ${rmse:.2f}")
    print(f"• R² (Coefficient of Determination): {r2:.3f}")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

# Evaluate on both training and test sets
train_metrics = calculate_metrics(y_train, y_train_pred, "Training Set")
test_metrics = calculate_metrics(y_test, y_test_pred, "Test Set")

# Check for overfitting
overfit_check = train_metrics['R2'] - test_metrics['R2']
if overfit_check > 0.1:
    print(f"\n⚠️ Potential overfitting detected (R² difference: {overfit_check:.3f})")
else:
    print(f"\n✅ Good generalization (R² difference: {overfit_check:.3f})")

## 2.3 Analyze Model Results

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🤖 Linear Regression Model Analysis', fontsize=16, fontweight='bold')

# 1. Actual vs Predicted (Test Set)
axes[0,0].scatter(y_test, y_test_pred, alpha=0.6, color='blue')
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,0].set_xlabel('Actual Price ($)')
axes[0,0].set_ylabel('Predicted Price ($)')
axes[0,0].set_title(f'Actual vs Predicted (R² = {test_metrics["R2"]:.3f})')
axes[0,0].grid(True, alpha=0.3)

# 2. Residuals plot
residuals = y_test - y_test_pred
axes[0,1].scatter(y_test_pred, residuals, alpha=0.6, color='green')
axes[0,1].axhline(y=0, color='r', linestyle='--')
axes[0,1].set_xlabel('Predicted Price ($)')
axes[0,1].set_ylabel('Residuals ($)')
axes[0,1].set_title('Residuals Plot')
axes[0,1].grid(True, alpha=0.3)

# 3. Feature importance (coefficients)
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'coefficient': model.coef_
}).sort_values('coefficient', key=abs, ascending=True)

axes[1,0].barh(range(len(feature_importance)), feature_importance['coefficient'], 
               color=['red' if x < 0 else 'blue' for x in feature_importance['coefficient']])
axes[1,0].set_yticks(range(len(feature_importance)))
axes[1,0].set_yticklabels(feature_importance['feature'])
axes[1,0].set_xlabel('Coefficient Value')
axes[1,0].set_title('Feature Importance (Coefficients)')
axes[1,0].grid(True, alpha=0.3)

# 4. Error distribution
axes[1,1].hist(residuals, bins=30, alpha=0.7, color='purple')
axes[1,1].axvline(x=0, color='r', linestyle='--')
axes[1,1].set_xlabel('Prediction Error ($)')
axes[1,1].set_ylabel('Frequency')
axes[1,1].set_title(f'Error Distribution (MAE = ${test_metrics["MAE"]:.2f})')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Feature importance insights
print("🎯 Model Insights:")
print(f"• Most important feature: {feature_importance.iloc[-1]['feature']} (coeff: {feature_importance.iloc[-1]['coefficient']:.3f})")
print(f"• Least important feature: {feature_importance.iloc[0]['feature']} (coeff: {feature_importance.iloc[0]['coefficient']:.3f})")
print(f"• Model intercept: ${model.intercept_:.2f}")
print(f"• Average prediction error: ${test_metrics['MAE']:.2f}")

## 2.4 Test Model with New Scenarios

In [ ]:
# Create test scenarios
test_scenarios = pd.DataFrame({
    'scenario': ['Rush Hour Peak', 'Weekend Casual', 'Event Day', 'Low Demand', 'Bad Weather'],
    'hour': [8, 14, 19, 11, 16],
    'day_of_week': [1, 6, 5, 2, 3],  # Monday, Sunday, Saturday, Wednesday, Thursday
    'is_weekend': [0, 1, 1, 0, 0],
    'occupancy_rate': [0.85, 0.45, 0.92, 0.25, 0.60],
    'queue_length': [8, 2, 12, 1, 4],
    'weather_score': [0.9, 0.8, 0.9, 0.7, 0.3],  # 0.3 = bad weather
    'event_factor': [0, 0, 1, 0, 0],
    'location_id': [2, 1, 4, 0, 3]  # Different zones
})

# Make predictions for test scenarios
scenario_features = test_scenarios[feature_columns]
scenario_predictions = model.predict(scenario_features)

# Display results
results_df = test_scenarios[['scenario']].copy()
results_df['predicted_price'] = np.round(scenario_predictions, 2)
results_df['occupancy'] = test_scenarios['occupancy_rate']
results_df['weather'] = test_scenarios['weather_score']
results_df['event'] = test_scenarios['event_factor'].map({0: 'No', 1: 'Yes'})

print("🧪 Model Predictions for Test Scenarios:")
print("=" * 60)
for idx, row in results_df.iterrows():
    print(f"📍 {row['scenario']}:")
    print(f"   💰 Predicted Price: ${row['predicted_price']:.2f}")
    print(f"   📊 Occupancy: {row['occupancy']:.1%}")
    print(f"   🌤️ Weather: {row['weather']:.1f}/1.0")
    print(f"   🎉 Event: {row['event']}")
    print()

# Business insights
print("💡 Business Insights:")
highest_price_idx = results_df['predicted_price'].idxmax()
lowest_price_idx = results_df['predicted_price'].idxmin()

print(f"• Highest price scenario: {results_df.loc[highest_price_idx, 'scenario']} (${results_df.loc[highest_price_idx, 'predicted_price']:.2f})")
print(f"• Lowest price scenario: {results_df.loc[lowest_price_idx, 'scenario']} (${results_df.loc[lowest_price_idx, 'predicted_price']:.2f})")
print(f"• Price range: ${results_df['predicted_price'].min():.2f} - ${results_df['predicted_price'].max():.2f}")
print(f"• Average predicted price: ${results_df['predicted_price'].mean():.2f}")

---
# ⚛️ Experiment 3: Quantum Circuit Simulation (30 minutes)

**Goal**: Create and run basic quantum circuits to understand quantum concepts

## 3.1 Simple Quantum Circuit

In [ ]:
# Create a simple 2-qubit quantum circuit
qc = QuantumCircuit(2, 2)

# Start with |00⟩ state (both qubits in 0)
print("🔬 Creating 2-qubit quantum circuit...")

# Apply Hadamard gate to first qubit (creates superposition)
qc.h(0)  # Now we have (|00⟩ + |10⟩)/√2

# Apply CNOT gate (creates entanglement)
qc.cx(0, 1)  # Now we have (|00⟩ + |11⟩)/√2 - Bell state!

# Add measurements
qc.measure([0, 1], [0, 1])

# Display the circuit
print("\n📊 Quantum Circuit:")
print(qc)

# Visualize the circuit (if possible)
try:
    qc.draw(output='mpl')
    plt.title('Simple Quantum Circuit')
    plt.show()
except:
    print("📝 Circuit visualization not available in text mode")

## 3.2 Run Quantum Simulation

In [ ]:
# Set up quantum simulator
simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit
print("🔄 Running quantum simulation...")
job = execute(qc, simulator, shots=1000)
result = job.result()
counts = result.get_counts(qc)

print(f"✅ Simulation complete! (1000 shots)")
print(f"\n📊 Measurement Results:")
for state, count in sorted(counts.items()):
    probability = count / 1000
    print(f"• State |{state}⟩: {count:3d} times ({probability:.1%})")

# Visualize results
try:
    plot_histogram(counts)
    plt.title('Quantum Measurement Results')
    plt.show()
except:
    # Manual bar plot if qiskit visualization fails
    states = list(counts.keys())
    frequencies = list(counts.values())
    
    plt.figure(figsize=(10, 6))
    plt.bar(states, frequencies, color='skyblue', alpha=0.8)
    plt.xlabel('Quantum State')
    plt.ylabel('Frequency')
    plt.title('Quantum Measurement Results (Bell State)')
    plt.grid(True, alpha=0.3)
    
    # Add probability labels
    for i, (state, freq) in enumerate(zip(states, frequencies)):
        plt.text(i, freq + 10, f'{freq/1000:.1%}', ha='center', va='bottom')
    
    plt.show()

# Explain the results
print("\n🧠 Understanding the Results:")
print("• We created a 'Bell State' - maximum entanglement between qubits")
print("• Expected: 50% |00⟩ and 50% |11⟩ (never |01⟩ or |10⟩)")
if '01' in counts or '10' in counts:
    error_rate = (counts.get('01', 0) + counts.get('10', 0)) / 1000
    print(f"• Small error rate: {error_rate:.1%} (due to quantum noise simulation)")
else:
    print("• Perfect entanglement observed! 🎉")
    
print("• This demonstrates quantum superposition and entanglement!")

## 3.3 Quantum Feature Encoding Experiment

In [ ]:
# Simple quantum feature encoding for parking data
def encode_parking_features(occupancy_rate, queue_length):
    """
    Encode parking features into quantum circuit
    
    - occupancy_rate: encoded as rotation angle
    - queue_length: encoded as conditional gates
    """
    
    # Create 2-qubit circuit
    qc = QuantumCircuit(2, 2)
    
    # Encode occupancy as rotation (0.0 → 0°, 1.0 → π)
    occupancy_angle = occupancy_rate * np.pi
    qc.ry(occupancy_angle, 0)
    
    # Encode queue length as conditional operations
    if queue_length > 5:  # High queue
        qc.x(1)  # Flip second qubit
    if queue_length > 10:  # Very high queue
        qc.cx(0, 1)  # Add entanglement
    
    # Measure
    qc.measure([0, 1], [0, 1])
    
    return qc

# Test with different parking scenarios
scenarios = [
    {"name": "Low Demand", "occupancy": 0.2, "queue": 1},
    {"name": "Medium Demand", "occupancy": 0.6, "queue": 5},
    {"name": "High Demand", "occupancy": 0.9, "queue": 12}
]

print("🧪 Quantum Feature Encoding Experiment:")
print("=" * 50)

results_summary = []

for scenario in scenarios:
    print(f"\n📍 Scenario: {scenario['name']}")
    print(f"   Occupancy: {scenario['occupancy']:.1%}")
    print(f"   Queue Length: {scenario['queue']}")
    
    # Create quantum circuit for this scenario
    qc_scenario = encode_parking_features(scenario['occupancy'], scenario['queue'])
    
    # Run simulation
    job = execute(qc_scenario, simulator, shots=100)
    result = job.result()
    counts = result.get_counts(qc_scenario)
    
    # Calculate quantum "price signal"
    # Higher probability of |11⟩ → higher price
    prob_11 = counts.get('11', 0) / 100
    quantum_price_signal = prob_11
    
    print(f"   Quantum States: {dict(counts)}")
    print(f"   Quantum Price Signal: {quantum_price_signal:.2f}")
    
    results_summary.append({
        'scenario': scenario['name'],
        'occupancy': scenario['occupancy'],
        'queue': scenario['queue'],
        'quantum_signal': quantum_price_signal,
        'states': dict(counts)
    })

# Compare quantum signals
print("\n📊 Quantum Signal Comparison:")
scenarios_df = pd.DataFrame(results_summary)
scenarios_df = scenarios_df.sort_values('quantum_signal')

plt.figure(figsize=(12, 6))
bars = plt.bar(scenarios_df['scenario'], scenarios_df['quantum_signal'], 
               color=['green', 'orange', 'red'], alpha=0.7)
plt.ylabel('Quantum Price Signal')
plt.title('Quantum Feature Encoding Results')
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, scenarios_df['quantum_signal']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.2f}', ha='center', va='bottom')

plt.show()

print("\n🎯 Quantum Encoding Insights:")
print(f"• Lowest signal: {scenarios_df.iloc[0]['scenario']} ({scenarios_df.iloc[0]['quantum_signal']:.2f})")
print(f"• Highest signal: {scenarios_df.iloc[-1]['scenario']} ({scenarios_df.iloc[-1]['quantum_signal']:.2f})")
print("• Quantum circuits can encode multiple features simultaneously!")
print("• Next week: We'll use this for actual quantum machine learning!")

---
# 📝 Week 1 Summary & Reflection

## 🎯 What We Learned

### **Machine Learning Concepts:**
1. **Problem Formulation**: Parking pricing as regression problem
2. **Feature Engineering**: Time, location, demand, and external factors
3. **Model Training**: Linear regression baseline
4. **Evaluation Metrics**: MAE, RMSE, R² for business context
5. **Model Interpretation**: Feature importance and business insights

### **Quantum Computing Concepts:**
1. **Quantum States**: Superposition and entanglement
2. **Quantum Gates**: H, X, CNOT for quantum operations
3. **Measurement**: Probabilistic outcomes
4. **Feature Encoding**: Mapping classical data to quantum states
5. **Quantum Advantage**: Parallel processing potential

## 🔍 Key Findings

### **From Our Data:**
- Occupancy rate is the strongest price predictor
- Rush hours and events significantly impact pricing
- Location differences create price premiums
- Weather and queue length provide additional signals

### **From Our Models:**
- Linear regression achieved reasonable baseline performance
- Model generalizes well (no severe overfitting)
- Feature importance aligns with business intuition
- Room for improvement with more complex models

### **From Quantum Experiments:**
- Successfully created and measured quantum states
- Demonstrated entanglement with Bell states
- Encoded parking features into quantum circuits
- Different scenarios produce distinct quantum signatures

## 🚀 Next Week Preparation

**Questions to Explore:**
1. How can quantum circuits improve upon classical ML?
2. What quantum algorithms are best for pricing optimization?
3. How do we measure quantum advantage quantitatively?

**Skills to Develop:**
1. Variational quantum circuits (VQC)
2. Quantum feature maps
3. Hybrid quantum-classical optimization
4. Performance benchmarking

Ready for Week 2? Let's dive deeper into quantum machine learning! 🎉